<a href="https://colab.research.google.com/github/marifemendesmtl/scam_escape/blob/main/scam_escape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instala google_generativeai

In [1]:
!pip install -U -q google-generativeai

In [2]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

Lê api_key de userdata

In [25]:
from google.colab import userdata
api_key = userdata.get('API_KEY')


In [26]:
# Configura a biblioteca do cliente fornecendo sua API_KEY.
genai.configure(api_key=api_key)

In [5]:
model = 'gemini-1.5-pro-latest' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}

In [6]:
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjowLCJtYXhfb3V0cHV0X3Rva2VucyI6ODE5Miwic3RvcF9zZXF1ZW5jZXMiOltdfQ==' # @param {isTemplate: true}

In [7]:
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}

In [8]:
contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

Apresentação do Chatbot LolE


In [27]:
apresentacao = """
Olá, tudo bem?
Eu sou LolE e estou aqui para te ajudar a escapar de enrrascadas, golpes e abusos em aplicativos de relacionamento.
"""

In [28]:
print(apresentacao)


Olá, tudo bem?
Eu sou LolE e estou aqui para te ajudar a escapar de enrrascadas em aplicativos de relacionamento.



In [19]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

In [24]:
prompt = input("Escreva o seu relato ou FIM para sair: ")

while prompt != "FIM":
  response = chat.send_message(prompt, stream=False )
  print("\nResposta: ", response.text, "\n")
  prompt = input("Escreva o seu relato ou FIM para sair: ")


Escreva o seu relato ou FIM para sair: FIM
